In [6]:
import numpy as np
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Libraries imported


In [7]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
print(soup.find('table'))

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

In [9]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields),3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_toronto= pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_toronto.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_toronto.head()

Postcode           Borough              Neighbourhood
0      M1A      Not assigned                           
1      M2A      Not assigned                           
2      M3A        North York                  Parkwoods
3      M4A        North York           Victoria Village
4      M5A  Downtown Toronto  Regent Park, Harbourfront

In [10]:
df_toronto1=df_toronto[df_toronto['Borough']=='Not assigned']
df_toronto.drop(df_toronto1.index,inplace=True)
df_toronto.head()

Postcode           Borough                                Neighbourhood
2      M3A        North York                                    Parkwoods
3      M4A        North York                             Victoria Village
4      M5A  Downtown Toronto                    Regent Park, Harbourfront
5      M6A        North York             Lawrence Manor, Lawrence Heights
6      M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [14]:
df_toronto.head()

Postcode           Borough                                Neighbourhood
2      M3A        North York                                    Parkwoods
3      M4A        North York                             Victoria Village
4      M5A  Downtown Toronto                    Regent Park, Harbourfront
5      M6A        North York             Lawrence Manor, Lawrence Heights
6      M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [12]:
df_torontofinal = df_toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_torontofinal.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_torontofinal

Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H        North York   
29       M3J        North York   
..       ...               ...   
73       M6C              York   
74       M6E              York   
75       M6G  Downtown Toronto   
76       M6H      West Toronto   
77       M6J      West Toronto   
78       M6K      West Toronto   
79       M6L        North York   
80       M6M              York   
81       M6N              York   
82       M6P      West Toronto   
83       M6R      West Toronto   
84       M6S      West Toronto   
85       M7A  Downtown Toronto   
86       M7R       Mississauga   
87       M7Y      East Toronto   
88       M8V         Etobicoke   
89       M8W         Etobicoke   
90       M8X         Etobicoke   
91       M8Y         Etobicoke   
92       M8Z         Etobicoke   
93       M9A         Etobicoke   
94       M9B         Etobicoke   
95       M9C         Etobicoke   
96       M9L        North York   
97       M9M        North York   
98       M9N              York   
99       M9P         Etobicoke   
100      M9R         Etobicoke   
101      M9V         Etobicoke   
102      M9W         Etobicoke   

                                         Neighbourhood  
0                                       Malvern, Rouge  
1               Rouge Hill, Port Union, Highland Creek  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
5                                  Scarborough Village  
6          Kennedy Park, Ionview, East Birchmount Park  
7                      Golden Mile, Clairlea, Oakridge  
8      Cliffside, Cliffcrest, Scarborough Village West  
9                          Birch Cliff, Cliffside West  
10   Dorset Park, Wexford Heights, Scarborough Town...  
11                                   Wexford, Maryvale  
12                                           Agincourt  
13             Clarks Corners, Tam O'Shanter, Sullivan  
14   Milliken, Agincourt North, Steeles East, L'Amo...  
15                       Steeles West, L'Amoreaux West  
16                                         Upper Rouge  
17                                   Hillcrest Village  
18                        Fairview, Henry Farm, Oriole  
19                                     Bayview Village  
20                            York Mills, Silver Hills  
21                             Willowdale, Newtonbrook  
22                                          Willowdale  
23                                     York Mills West  
24                                          Willowdale  
25                                           Parkwoods  
26                                           Don Mills  
27                                           Don Mills  
28     Bathurst Manor, Wilson Heights, Downsview North  
29                     Northwood Park, York University  
..                                                 ...  
73                                  Humewood-Cedarvale  
74            

In [13]:
df_toronto.shape

(103, 3)